In [8]:
from openai import OpenAI

client = OpenAI()

# データの準備

普通に投稿を生成すると自画自賛の鼻につく文章が生成される。

客観的で好ましい文章を生成するアプローチとして以下を比較したい。

- プロンプトチューニング
- ファインチューニング

それぞれ、コスト、品質の面で評価する。

In [5]:
import pandas as pd

df = pd.read_csv("csv/焼肉きんぐ口コミセンテンス.csv")
filtered_df = df[(df['sentiment'] == 'POSITIVE') & (df['rating'] >= 3.0)]
filtered_df

,review_id,review_sentence_id,store_id,store_name,title,sentence,reviewer_name,rating,sentiment,posted_at
0,194238217,198167671,186406,焼肉きんぐ 羽根店,NaN,他国の店員さんもいましたがその方たちの接客はそれなりに良いと思いました。,funkyboy,3.0,POSITIVE,2025-02-11 16:49:51.3082
1,194238032,198167315,186000,焼肉きんぐ 那覇久茂地店,NaN,安くてデカ盛りのローストポーク食べ放題,林林,5.0,POSITIVE,2025-02-11 16:33:17.807767
2,194238032,198167317,186000,焼肉きんぐ 那覇久茂地店,NaN,2回以上のご来店をお勧めします,林林,5.0,POSITIVE,2025-02-11 16:33:17.807767
3,194238216,198167789,186407,焼肉きんぐ 蒲郡店,NaN,値段と品目、味には非常に満足しています。,三浦寛樹,4.0,POSITIVE,2025-02-11 15:45:41.585355
4,194238216,198167791,186407,焼肉きんぐ 蒲郡店,NaN,総合的にまた行こうと思えるくらいの満足度です。,三浦寛樹,4.0,POSITIVE,2025-02-11 15:45:41.585355
...,...,...,...,...,...,...,...,...,...,...
987,188231750,184920581,186283,焼肉きんぐ 神戸学院前店,NaN,レイアウトが効率化された分、座席数が増えて、並ばなくても入店できるようになったようです。,Wasabi Free,4.0,POSITIVE,2025-01-13 08:00:40.669633
988,188231750,184920583,186283,焼肉きんぐ 神戸学院前店,NaN,焼肉食べ放題も日々進化しているのだと感心しました。,Wasabi Free,4.0,POSITIVE,2025-01-13 08:00:40.669633
989,188231750,184920584,186283,焼肉きんぐ 神戸学院前店,NaN,お肉のお味は安定の焼肉きんぐです。,Wasabi Free,4.0,POSITIVE,2025-01-13 08:00:40.669633
990,188231750,184920585,186283,焼肉きんぐ 神戸学院前店,NaN,５大名物は時々入れ替えがありますが、きんぐカルビがお肉の味が濃厚で一番好きです。,Wasabi Free,4.0,POSITIVE,2025-01-13 08:00:40.669633


In [7]:
keywords = ['カルビ', '接客', '席']
sentences_by_keyword = {}

for keyword in keywords:
    keyword_df = filtered_df[filtered_df['sentence'].str.contains(keyword, na=False)]
    keyword_sentences = keyword_df['sentence'].head(10).tolist()
    sentences_by_keyword[keyword] = keyword_sentences

sentences_by_keyword

{'カルビ': ['キングカルビ、炙りすき焼きカルビ、冷麺、韓国のり卵ご飯など、どれも美味しかったです。',
  'タンやカルビ、ロースやハラミ等、一通りのお肉が食べ放題てす。',
  '気を取り直し、今回はきんぐコースにして、前回美味しかったきんぐカルビを集中攻撃！',
  'きんぐカルビ最高！',
  '食べ放題の焼肉安かろう悪かろうと思い近づかなかったがきんぐカルビは肉肉しくてなかなかうまい',
  'キングカルビ美味しかった。',
  'お肉はカルビが食べやすかったです。',
  'きんぐカルビは男性子供にはジューシーで好評でした。',
  '個人的に薄めの黒毛和牛（カルビかな？）がおいしかったです。',
  'きんぐコースを頼みましたがきんぐカルビが大きくて、しかも美味しい！！'],
 '接客': ['他国の店員さんもいましたがその方たちの接客はそれなりに良いと思いました。',
  '接客がどうこうというよりは、どちらかというと、一生懸命働いているスタッフ達というイメージです♪',
  'ここはスタッフがとてもテキパキしていて、とても接客がすごいなって思います。',
  '忙しい時間帯でも、丁寧な説明、接客で気持ちよく過ごせました。',
  '明るい店員さんばかりで終始接客も丁寧で、',
  'アルバイトのレベルも高く、その辺の高級店よりも接客態度がいい。',
  '接客、雰囲気を含め太田店より良かったです。',
  '接客が異常に良いし',
  '店員さんの接客がすご過ぎて驚く！！',
  '焼肉ポリスの店員さんの接客が丁寧で色んな食べ方を教えてくれるので凄く楽しく食事できた'],
 '席': ['当日は予約時間より早くチェックインしましたが、空いていたのですぐに席に案内してくれました。',
  '早めに予約すればすんなり入れ、いい席に座れます。',
  '環境も綺麗で席も広く、食材の種類も豊富で最高です。',
  '事前予約してお邪魔したので、スムーズに席へ着く事ができました。',
  '前日予約していたので、入店後スムーズに席に案内してくれました💡',
  '家族連れにはもってこいのボックス席でゆったりできます。',
  '店内は満席大盛況、スタッフもかなり多い。',
  '隣の席の子供に対しての対応がまるでNHKのお兄さん然としたもので驚きと共に感

## 普通に投稿文を作成

In [22]:
import pandas as pd
import json

def calculate_cost(model, prompt_tokens, completion_tokens):
    if model == "gpt-4o-mini":
        prompt_cost_per_1M = 0.15  # $0.15 per 1M input tokens
        completion_cost_per_1M = 0.075  # $0.075 per 1M output tokens
    elif model == "gpt-4o":
        prompt_cost_per_1M = 2.5  # $2.5 per 1M input tokens
        completion_cost_per_1M = 1.25  # $1.25 per 1M output tokens
    else:
        # デフォルト値（不明なモデルの場合）
        prompt_cost_per_1M = 0.15
        completion_cost_per_1M = 0.075
    
    # 1Mトークンあたりの価格を計算
    prompt_cost = (prompt_tokens / 1000000) * prompt_cost_per_1M
    completion_cost = (completion_tokens / 1000000) * completion_cost_per_1M
    
    return prompt_cost + completion_cost

def system_prompt():
    prompt = [
        'ある企業の口コミのリストを提供します。',
        '口コミから、利用者に高く評価されているポイントをひとつだけ抽出し、それをPRする投稿を作成してください。',
        '投稿文は、MEO対策を考慮し、"出力ルール"に従って作成してください。\n',
        '# 出力ルール',
        '- 文章は日本語',
        '- 100文字以上200文字以内',
        '- タイトル不要',
        '- 文頭、文末に括弧不要、文章のみ出力',
    ]
    return "\n".join(prompt)

def generate_post(keyword, sentences, model="gpt-4o"):
    sentences_text = "\n".join([f"- {sentence}" for sentence in sentences])
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt()},
            {"role": "user", "content": sentences_text},
        ],
    )

    post_text = completion.choices[0].message.content

    prompt_tokens = completion.usage.prompt_tokens
    completion_tokens = completion.usage.completion_tokens
    cost = calculate_cost(model, prompt_tokens, completion_tokens)
    print(cost)
    
    return post_text, cost

results = []

# 各キーワードについて10件の投稿を生成
for keyword, sentences in sentences_by_keyword.items():
    for i in range(10):
        post_text, cost = generate_post(keyword, sentences)
        
        results.append({
            "調整方法": "未調整",
            "口コミテーマ": keyword,
            "投稿文": post_text,
            "コスト": cost
        })

results_df = pd.DataFrame(results)
results_df.to_csv("csv/投稿文自動生成スタイル調整_未調整.csv", index=False)

0.0010137500000000001
0.001015
0.00101125
0.00100875
0.0010525
0.000995
0.0010350000000000001
0.00100125
0.0010575
0.0010175
0.001065
0.0010400000000000001
0.00108875
0.0010675
0.00100375
0.00104875
0.0010125
0.0010287500000000002
0.00104125
0.0010525
0.00106
0.00109625
0.0010775
0.00106125
0.00104
0.0010624999999999999
0.00109125
0.00107625
0.00105625
0.0010624999999999999


In [23]:
results_df

,調整方法,口コミテーマ,投稿文,コスト
0,未調整,カルビ,ジューシーで肉肉しい「きんぐカルビ」が大好評！キンカルの魅力を存分に味わえる食べ放題コースで...,0.001014
1,未調整,カルビ,焼肉きんぐの「きんぐカルビ」が大好評！ジューシーで肉厚、食べ放題でも妥協しないその美味しさは...,0.001015
2,未調整,カルビ,当社では、特に「きんぐカルビ」の美味しさが多くのお客様に高く評価されています。ジューシーで食...,0.001011
3,未調整,カルビ,キングカルビの美味しさをぜひ体験してください！この絶品カルビは食べ放題でも大人気。ジューシー...,0.001009
4,未調整,カルビ,きんぐカルビが大好評！ジューシーで肉肉しい味わいが、多くのお客様に選ばれています。食べ放題プ...,0.001053
5,未調整,カルビ,ジューシーで肉厚な「きんぐカルビ」は、焼肉ファン必食の一品！食べ放題で思い切り楽しめ、お子様...,0.000995
6,未調整,カルビ,キングカルビが大人気！良質なお肉の旨みが口いっぱいに広がり、ジューシーな味わいを堪能できます...,0.001035
7,未調整,カルビ,当店の「きんぐカルビ」は、そのジューシーさと肉々しさで多くの方に愛されています。食べ放題で心...,0.001001
8,未調整,カルビ,きんぐカルビはジューシーで肉肉しい味わいが大好評！食べ放題で楽しめるので、家族や友人と一緒に...,0.001058
9,未調整,カルビ,ジューシーで美味しいと大好評の「きんぐカルビ」！焼肉の食べ放題で安くても品質は譲らず、特にき...,0.001017


## プロンプトで調整

追加した指示:
- 口コミで言及された点の羅列ではなく、投稿文全体がひとつのテーマに沿う自然な構成になるようにしてください
- 定性的な評価は、主観ではなく、そういった口コミや感想をいただいているという事実として記載してください
- 「工夫によって良い体験を提供できている」という言い回しではなく、「良い体験を提供するためにこのような工夫をしている」という言い回しをしてください
  - 悪い例: 「テキパキとした接客と丁寧な説明でお客様に快適な時間をご提供しています」
  - 良い例: 「お客様に快適な時間をご提供するために、テキパキとした接客と丁寧な説明を徹底しています」

In [24]:
# ... 既存コード ...

def adjusted_system_prompt():
    prompt = [
        'ある企業の口コミのリストを提供します。',
        '口コミから、利用者に高く評価されているポイントをひとつだけ抽出し、それをPRする投稿を作成してください。',
        '投稿文は、MEO対策を考慮し、"出力ルール"、"言い回しの注意"に従って作成してください。\n',
        '# 出力ルール',
        '- 文章は日本語',
        '- 100文字以上200文字以内',
        '- タイトル不要',
        '- 文頭、文末に括弧不要、文章のみ出力\n',
        '# 言い回しの注意',
        '- 口コミで言及された点を羅列するだけの不自然な投稿文にしないこと',
        '- 投稿文全体がひとつのテーマに沿った自然な構成とすること',
        '- 主観的な定性的評価は書かず、そういった口コミや感想をいただいたという事実のみ記載すること',
        '- 「工夫の結果、サービスの向上につながった」という言い回しを避け、「サービス向上のため、工夫をしている」という言い回しにすること',
        '  - 悪い例: 「テキパキとした接客と丁寧な説明でお客様に快適な時間をご提供しています」',
        '  - 良い例: 「お客様に快適な時間をご提供するために、テキパキとした接客と丁寧な説明を徹底しています」',
    ]
    return "\n".join(prompt)

def generate_adjusted_post(keyword, sentences, model="gpt-4o"):
    sentences_text = "\n".join([f"- {sentence}" for sentence in sentences])
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": adjusted_system_prompt()},
            {"role": "user", "content": sentences_text},
        ],
    )

    post_text = completion.choices[0].message.content

    prompt_tokens = completion.usage.prompt_tokens
    completion_tokens = completion.usage.completion_tokens
    cost = calculate_cost(model, prompt_tokens, completion_tokens)
    print(cost)
    
    return post_text, cost

adjusted_results = []

# 各キーワードについて10件の投稿を生成
for keyword, sentences in sentences_by_keyword.items():
    for i in range(10):
        post_text, cost = generate_adjusted_post(keyword, sentences)
        
        adjusted_results.append({
            "調整方法": "プロンプト調整",
            "口コミテーマ": keyword,
            "投稿文": post_text,
            "コスト": cost
        })

adjusted_results_df = pd.DataFrame(adjusted_results)
adjusted_results_df.to_csv("csv/投稿文自動生成スタイル調整_プロンプト.csv", index=False)

adjusted_results_df

0.0016125
0.00155625
0.0016025
0.0015925
0.00156625
0.0015687499999999998
0.00155625
0.0015787499999999999
0.0015825
0.00161375
0.001585
0.00159625
0.0015899999999999998
0.0015737499999999998
0.00163875
0.00161375
0.0016275
0.00158625
0.0015875
0.00163
0.0016587499999999999
0.0016224999999999998
0.0016612499999999997
0.0016449999999999998
0.00164125
0.0016524999999999999
0.0016512499999999997
0.0016374999999999998
0.0016374999999999998
0.0016274999999999998


,調整方法,口コミテーマ,投稿文,コスト
0,プロンプト調整,カルビ,食べ放題の焼肉専門店で、特にきんぐカルビが好評をいただいております！ジューシーで食べ応えのあ...,0.001613
1,プロンプト調整,カルビ,"サービス向上のため、お客様に大変ご好評いただいている""きんぐカルビ""に注力しています。ジュー...",0.001556
2,プロンプト調整,カルビ,サービス向上のため、私たちはお客様からいただいたご意見を大切にしています。多くの方から人気を...,0.001602
3,プロンプト調整,カルビ,「きんぐカルビ」に対する高評価の口コミが多く寄せられています。当店では、サービス向上のため、...,0.001592
4,プロンプト調整,カルビ,当店の「きんぐカルビ」は、多くのお客様から美味しいとの声をいただいております。ジューシーさと...,0.001566
5,プロンプト調整,カルビ,当店の大人気メニュー「きんぐカルビ」は、食べ放題でありながらその質が高く、多くのお客様から美...,0.001569
6,プロンプト調整,カルビ,当店では、特に「きんぐカルビ」が多くのお客様からおいしいとのご評価をいただいております。肉肉...,0.001556
7,プロンプト調整,カルビ,「きんぐカルビ」は多くの口コミで高評価をいただいています。サービス向上のため、ジューシーでボ...,0.001579
8,プロンプト調整,カルビ,お客様から「きんぐカルビが美味しい」と多数のお声をいただいております。カルビがジューシーで食...,0.001582
9,プロンプト調整,カルビ,焼肉キングで大人気の「キングカルビ」は、多くのお客様に美味しいとご好評いただいています。特に...,0.001614


## fine-tuning

In [25]:
import pandas as pd

df = pd.read_csv("csv/finetuning_train.csv")
df.head()

,completion,prompt
0,10/21～10/31まで✨\n【アプリ会員様限定】店舗限定！トマオニオリジナルクリアファイ...,NaN
1,".\nユブチョバプランチ\n1,080円(税込1,188円)\n\n人気のユブチョバプがラン...",NaN
2,【クリスマスイブまでカウントダウン あと10日】\n12月の最大のイベントのクリスマスイブま...,NaN
3,10月16日～配布開始！\n削って当てよう！\n創業祭スポーツニクラッチ⚽\n\nご来店のお...,NaN
4,ラグナガーデンホテルでは、10月1日からクリスマスケーキの〈予約販売〉を開始いたしました！\...,NaN


In [51]:
import json
from pydantic import BaseModel

class Judge(BaseModel):
  use: bool

df['use'] = False

system_prompt = """
テキストが提供されるので、そのテキストが以下の条件に当てはまるか判定してください。

条件:
1. 日本語で書かれている（商品名など部分的な外国語はOK）。
2. 既存のサービスや商品をPRする内容になっている。
3. イベントやアクセス、新店舗情報等の告知ではない。
"""

# for idx, row in df.iterrows():
#     text = row['completion']
    
#     try:
#         completion = client.beta.chat.completions.parse(
#             model="gpt-4o-mini",
#             messages=[
#                 {"role": "system", "content": system_prompt},
#                 {"role": "user", "content": text},
#             ],
#             response_format=Judge,
#         )
#         result = completion.choices[0].message.parsed.use
#         print(f"Row {idx} の判定結果: {result}")
#         df.at[idx, 'use'] = result
#     except Exception as e:
#         # エラー発生時でも次の行へ進める
#         print(f"Row {idx} の処理中にエラー発生: {e}")
#         continue

In [33]:
# df.to_csv("csv/finetuning_train_judge.csv", index=False)
# df.head(10)

,completion,prompt,use
0,10/21～10/31まで✨\n【アプリ会員様限定】店舗限定！トマオニオリジナルクリアファイ...,NaN,False
1,".\nユブチョバプランチ\n1,080円(税込1,188円)\n\n人気のユブチョバプがラン...",NaN,True
2,【クリスマスイブまでカウントダウン あと10日】\n12月の最大のイベントのクリスマスイブま...,NaN,False
3,10月16日～配布開始！\n削って当てよう！\n創業祭スポーツニクラッチ⚽\n\nご来店のお...,NaN,False
4,ラグナガーデンホテルでは、10月1日からクリスマスケーキの〈予約販売〉を開始いたしました！\...,NaN,True
5,10月的第三個星期日，是日本的「孫子日」。\n在這個加深和孫子之前親密關係的特別日子裡，何不...,NaN,True
6,10月の第3日曜日は、孫の日。\nお孫様との絆を深める特別な日に、『ゆず庵』の「お祝いコース...,NaN,True
7,🍴11～14時のランチがオトク🍴\n\nキャンペーン実施期間中に\n本人確認済かつPayPa...,NaN,False
8,11/15~11/23まで✨\n【アプリ会員様限定】七五三＆勤労感謝クーポン配信😋\n\n七...,NaN,True
9,.\n\n11月29日は年に一度の＜いい肉の日＞🥩\n今年ラストの食べ放題を開催いたします🎉...,NaN,False


In [34]:
df = pd.read_csv("csv/finetuning_train_judge.csv")
df.head()

,completion,prompt,use
0,10/21～10/31まで✨\n【アプリ会員様限定】店舗限定！トマオニオリジナルクリアファイ...,NaN,False
1,".\nユブチョバプランチ\n1,080円(税込1,188円)\n\n人気のユブチョバプがラン...",NaN,True
2,【クリスマスイブまでカウントダウン あと10日】\n12月の最大のイベントのクリスマスイブま...,NaN,False
3,10月16日～配布開始！\n削って当てよう！\n創業祭スポーツニクラッチ⚽\n\nご来店のお...,NaN,False
4,ラグナガーデンホテルでは、10月1日からクリスマスケーキの〈予約販売〉を開始いたしました！\...,NaN,True


In [37]:
df['use'].sum()

747

In [49]:
import json
from pydantic import BaseModel

class PromptResponse(BaseModel):
  prompt: str

df['prompt'] = None

prompt_system = """
SNS投稿された文章を提供します。
その文章をLLMで自動生成する場合、どのようなpromptにすればよいか考え、出力してください。
ただし、promptには以下のような内容を含めてください。
- MEO対策を考慮した投稿文を作成する指示
- 文章は日本語とする指示
- 文字数は100文字以上200文字以内とする指示
- そのほか、completionから読み取れる具体的なドメイン知識
"""

# for idx, row in df.iterrows():
#     if not row['use']:
#         continue

#     text = row['completion']
    
#     try:
#         prompt_completion = client.beta.chat.completions.parse(
#             model="gpt-4o-mini",
#             messages=[
#                 {"role": "system", "content": prompt_system},
#                 {"role": "user", "content": text},
#             ],
#             response_format=PromptResponse,
#         )
#         prompt_text = prompt_completion.choices[0].message.parsed.prompt
#         print(f"Row {idx} の生成された prompt: {prompt_text}")
#         df.at[idx, 'prompt'] = prompt_text
#     except Exception as e:
#         print(f"Row {idx} の prompt 生成中にエラー発生: {e}")
#         continue

In [39]:
# df.to_csv("csv/finetuning_train_prompt.csv", index=False)
# df.head(10)

,completion,prompt,use
0,10/21～10/31まで✨\n【アプリ会員様限定】店舗限定！トマオニオリジナルクリアファイ...,None,False
1,".\nユブチョバプランチ\n1,080円(税込1,188円)\n\n人気のユブチョバプがラン...",MEO対策を考慮した投稿文を作成してください。内容は日本語で、文字数は100文字以上200文...,True
2,【クリスマスイブまでカウントダウン あと10日】\n12月の最大のイベントのクリスマスイブま...,None,False
3,10月16日～配布開始！\n削って当てよう！\n創業祭スポーツニクラッチ⚽\n\nご来店のお...,None,False
4,ラグナガーデンホテルでは、10月1日からクリスマスケーキの〈予約販売〉を開始いたしました！\...,次の指示に従って、MEO対策を考慮したSNS投稿文を自動生成してください。以下の条件を守って...,True
5,10月的第三個星期日，是日本的「孫子日」。\n在這個加深和孫子之前親密關係的特別日子裡，何不...,MEO対策を考慮した、日本語でのSNS投稿文を自動生成してください。内容は、10月の第3日曜...,True
6,10月の第3日曜日は、孫の日。\nお孫様との絆を深める特別な日に、『ゆず庵』の「お祝いコース...,MEO対策を考慮し、100文字以上200文字以内の日本語で、特定の飲食店のプロモーション文を...,True
7,🍴11～14時のランチがオトク🍴\n\nキャンペーン実施期間中に\n本人確認済かつPayPa...,None,False
8,11/15~11/23まで✨\n【アプリ会員様限定】七五三＆勤労感謝クーポン配信😋\n\n七...,MEO対策を考慮して、アプリ会員向けに七五三と勤労感謝の特別クーポンについての投稿文を日本語...,True
9,.\n\n11月29日は年に一度の＜いい肉の日＞🥩\n今年ラストの食べ放題を開催いたします🎉...,None,False


In [ ]:
df.to_csv("csv/finetuning_train_prompt.csv", index=False)
df.head(10)

### 訓練

In [8]:
import pandas as pd

df = pd.read_csv("csv/finetuning_train_prompt.csv")
train_df = df[(df.use == True) & (df.prompt.notnull())]
print(len(train_df))
train_df.head()

747


,completion,prompt,use
1,".\nユブチョバプランチ\n1,080円(税込1,188円)\n\n人気のユブチョバプがラン...",MEO対策を考慮した投稿文を作成してください。内容は日本語で、文字数は100文字以上200文...,True
4,ラグナガーデンホテルでは、10月1日からクリスマスケーキの〈予約販売〉を開始いたしました！\...,次の指示に従って、MEO対策を考慮したSNS投稿文を自動生成してください。以下の条件を守って...,True
5,10月的第三個星期日，是日本的「孫子日」。\n在這個加深和孫子之前親密關係的特別日子裡，何不...,MEO対策を考慮した、日本語でのSNS投稿文を自動生成してください。内容は、10月の第3日曜...,True
6,10月の第3日曜日は、孫の日。\nお孫様との絆を深める特別な日に、『ゆず庵』の「お祝いコース...,MEO対策を考慮し、100文字以上200文字以内の日本語で、特定の飲食店のプロモーション文を...,True
8,11/15~11/23まで✨\n【アプリ会員様限定】七五三＆勤労感謝クーポン配信😋\n\n七...,MEO対策を考慮して、アプリ会員向けに七五三と勤労感謝の特別クーポンについての投稿文を日本語...,True


In [50]:
train_dict = train_df[['prompt', 'completion']].to_dict(orient="records")

In [24]:
import openai
import json
import pandas as pd

# 1. トレーニングデータの作成例
# data_path = "json/training_data.jsonl"
# train_dict = train_df[['prompt', 'completion']].to_dict(orient="records")
# with open(data_path, "w", encoding="utf-8") as f:
#     for example in train_dict:
#         f.write(json.dumps(example, ensure_ascii=False) + "\n")


# 訓練データは、使用するモデルによりチャット形式か補完形式か異なる
# 以下はチャット形式の訓練データ作成
# https://platform.openai.com/docs/api-reference/fine-tuning/chat-input
data_path = "json/training_data.jsonl"
train_dict = train_df[['prompt', 'completion']].to_dict(orient="records")

with open(data_path, "w", encoding="utf-8") as f:
    for example in train_dict:
        new_example = {
            "messages": [
                {"role": "user", "content": example["prompt"]},
                {"role": "assistant", "content": example["completion"]}
            ]
        }
        f.write(json.dumps(new_example, ensure_ascii=False) + "\n")

In [25]:
import json
from collections import defaultdict

# トレーニングデータの検証
data_path = "json/training_data.jsonl"

# JSONLファイルの読み込み
with open(data_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

# 初期統計
print("Num examples:", len(dataset))
print("First example:")
print(dataset[0])

# フォーマットエラーチェック
format_errors = defaultdict(int)

for ex in dataset:
    # 各行が辞書であるか？
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    # "prompt" キーの存在チェック
    if "prompt" not in ex:
        format_errors["missing_prompt_key"] += 1
    # "completion" キーの存在チェック
    if "completion" not in ex:
        format_errors["missing_completion_key"] += 1

    # キーが存在する場合、値が文字列であるかチェック
    if "prompt" in ex and not isinstance(ex["prompt"], str):
        format_errors["prompt_not_string"] += 1
    if "completion" in ex and not isinstance(ex["completion"], str):
        format_errors["completion_not_string"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

Num examples: 747
First example:
{'messages': [{'role': 'user', 'content': 'MEO対策を考慮した投稿文を作成してください。内容は日本語で、文字数は100文字以上200文字以内でお願いします。具体的には、ユブチョバプランチの魅力を伝え、提供するメニュー（海老バジル、サーモン、ツナマヨ、プルコギ）、スープやサラダ、キムチ、黒カレー＆ライスバーの充実ぶりについて説明してください。また、限定の提供期間（12/6まで）も記載してください。'}, {'role': 'assistant', 'content': '.\nユブチョバプランチ\n1,080円(税込1,188円)\n\n人気のユブチョバプがランチでも食べられる！\n\n海老バジル🦐\nサーモン🐟\nツナマヨ🥫\nプルコギ🍖\nこの4種類が一度で食べられる、オトクなランチ🍴\n\n牛肉や牛骨を煮込んだ、コムタンスープ付き🐄\n\nさ・ら・に！\nサラダ、キムチと黒カレー＆ライスバー、スープバー付き！！\n黒カレー＆ライスとスープはおかわり自由です🍛\n\n12/6までの限定ランチです🌟\n\n じゅうじゅうカルビ  じゅうカル  食べ放題  焼肉食べ放題\n 韓国フェア  ユブチョバプ  いなり寿司  sushi  🍣\n ランチ  焼肉ランチ  おかわり自由'}]}
Found errors:
missing_prompt_key: 747
missing_completion_key: 747


In [26]:
import json
import tiktoken  # pip install tiktoken
import numpy as np
from collections import defaultdict

# tiktoken のエンコーディングを取得
encoding = tiktoken.get_encoding("cl100k_base")

# 各例（1レコード）からトークン数をカウントする関数
# ※ここでは、各フィールドごとに若干のオーバーヘッド (tokens_per_field) を加算し、
# さらに全体で3トークンを追加する簡易計算としています。
def num_tokens_from_example(example, tokens_per_field=3):
    num_tokens = 0
    # "prompt" と "completion" の両方をカウントする
    for key in ["prompt", "completion"]:
        if key in example and isinstance(example[key], str):
            num_tokens += tokens_per_field  # フィールドごとのオーバーヘッド
            num_tokens += len(encoding.encode(example[key]))
        else:
            # 該当キーが無い場合はカウントエラーとして扱う（ここでは0とする）
            num_tokens += 0
    num_tokens += 3  # 全体のオーバーヘッド
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)} / {max(values)}")
    print(f"mean / median: {np.mean(values):.2f} / {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.05)} / {np.quantile(values, 0.95)}")

# JSONL ファイルのパス（先ほど作成したファイル）
data_path = "json/training_data.jsonl"

# データセット読み込み
with open(data_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

print("Num examples:", len(dataset))
print("First example:")
print(dataset[0])

# 各例のトークン数を計算
example_token_counts = [num_tokens_from_example(ex) for ex in dataset]

print_distribution(example_token_counts, "tokens per example")

# 請求対象のトークン数の見積もり（例: 16,385 トークンの上限を適用）
MAX_TOKENS_PER_EXAMPLE = 16385

# デフォルトの学習エポック数の推定条件
TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_train_examples = len(dataset)
n_epochs = TARGET_EPOCHS
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

# 各例について、上限トークン数と実際のトークン数の小さい方を合算
n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, token_count) for token_count in example_token_counts)

print(f"\nDataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Num examples: 747
First example:
{'messages': [{'role': 'user', 'content': 'MEO対策を考慮した投稿文を作成してください。内容は日本語で、文字数は100文字以上200文字以内でお願いします。具体的には、ユブチョバプランチの魅力を伝え、提供するメニュー（海老バジル、サーモン、ツナマヨ、プルコギ）、スープやサラダ、キムチ、黒カレー＆ライスバーの充実ぶりについて説明してください。また、限定の提供期間（12/6まで）も記載してください。'}, {'role': 'assistant', 'content': '.\nユブチョバプランチ\n1,080円(税込1,188円)\n\n人気のユブチョバプがランチでも食べられる！\n\n海老バジル🦐\nサーモン🐟\nツナマヨ🥫\nプルコギ🍖\nこの4種類が一度で食べられる、オトクなランチ🍴\n\n牛肉や牛骨を煮込んだ、コムタンスープ付き🐄\n\nさ・ら・に！\nサラダ、キムチと黒カレー＆ライスバー、スープバー付き！！\n黒カレー＆ライスとスープはおかわり自由です🍛\n\n12/6までの限定ランチです🌟\n\n じゅうじゅうカルビ  じゅうカル  食べ放題  焼肉食べ放題\n 韓国フェア  ユブチョバプ  いなり寿司  sushi  🍣\n ランチ  焼肉ランチ  おかわり自由'}]}

#### Distribution of tokens per example:
min / max: 3 / 3
mean / median: 3.00 / 3.0
p5 / p95: 3.0 / 3.0

Dataset has ~2241 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~6723 tokens


In [27]:
(6723 / (1000 * 1000)) * 25

0.168075

In [28]:
# ref: https://platform.openai.com/docs/api-reference/files/create?lang=python
# from openai import OpenAI
# client = OpenAI()

# response = client.files.create(
#   file=open(data_path, "rb"),
#   purpose="fine-tune"
# )
# response

# FileObject(id='file-W6vQgPEgFePUCVWA1TeGAh', bytes=1012538, created_at=1741054552, filename='training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)

FileObject(id='file-W6vQgPEgFePUCVWA1TeGAh', bytes=1012538, created_at=1741054552, filename='training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)

In [31]:
from openai import OpenAI

client = OpenAI()

job = client.fine_tuning.jobs.create(
    training_file='file-W6vQgPEgFePUCVWA1TeGAh',
    model="gpt-4o-2024-08-06",
    method={
        "type": "supervised",
    },
)
job

FineTuningJob(id='ftjob-t9qusnXOsDcnTyeNpaAoqBk4', created_at=1741055301, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-w90I6oahYVCKhyamyjvM9f32', result_files=[], seed=801833843, status='validating_files', trained_tokens=None, training_file='file-W6vQgPEgFePUCVWA1TeGAh', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None, metadata=None)

In [32]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-t9qusnXOsDcnTyeNpaAoqBk4', created_at=1741055301, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-w90I6oahYVCKhyamyjvM9f32', result_files=[], seed=801833843, status='validating_files', trained_tokens=None, training_file='file-W6vQgPEgFePUCVWA1TeGAh', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix=None, metadata=None), FineTuningJob(id='ftjob-s836REl9kigJuxETDNJDw8Co', created_at=1741054574, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(bat

In [ ]:
job = client.fine_tuning.jobs.retrieve(job_id)
print("Current status:", job.status)

In [ ]:
# 'ftjob-AuRv6B97tjt57F8tTdvYkVao' エラー
# 'ftjob-s836REl9kigJuxETDNJDw8Co': 重複
# 'ftjob-t9qusnXOsDcnTyeNpaAoqBk4': 最新

In [36]:
# キャンセルジョブの job_id（ハードコード）
job_ids_to_cancel = [
    'ftjob-t9qusnXOsDcnTyeNpaAoqBk4',  # 重複しているジョブ(新しい方)
]

for job_id in job_ids_to_cancel:
    try:
        cancelled_job = client.fine_tuning.jobs.cancel(job_id)
        print(f"Job {job_id} has been cancelled. New status: {cancelled_job.status}")
    except Exception as e:
        print(f"Error cancelling job {job_id}: {e}")

Job ftjob-t9qusnXOsDcnTyeNpaAoqBk4 has been cancelled. New status: cancelled


In [40]:
# 削除するジョブの job_id
job_ids_to_cancel = [
    'ftjob-AuRv6B97tjt57F8tTdvYkVao',
    'ftjob-t9qusnXOsDcnTyeNpaAoqBk4',
]

for job_id in job_ids_to_cancel:
    try:
        deleted_job = client.fine_tuning.jobs.delete(job_id)
        print(f"Job {job_id} has been deleted. New status: {deleted_job.status}")
    except Exception as e:
        print(f"Error deleting job {job_id}: {e}")

Error deleting job ftjob-AuRv6B97tjt57F8tTdvYkVao: 'Jobs' object has no attribute 'delete'
Error deleting job ftjob-t9qusnXOsDcnTyeNpaAoqBk4: 'Jobs' object has no attribute 'delete'


In [45]:
client.fine_tuning.jobs.retrieve("ftjob-s836REl9kigJuxETDNJDw8Co")

FineTuningJob(id='ftjob-s836REl9kigJuxETDNJDw8Co', created_at=1741054574, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:mov::B7DJJtvq', finished_at=1741059159, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-w90I6oahYVCKhyamyjvM9f32', result_files=['file-5UfeUNbFZCbdMUUJFZ8aMb'], seed=1013469261, status='succeeded', trained_tokens=763662, training_file='file-W6vQgPEgFePUCVWA1TeGAh', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix=None, metadata=None)

estimated_finishによると`2025年03月05日 01:56:15 (JST)`に完了予定。
完了したらfine_tuned_modelが返るらしい。

In [48]:
import pandas as pd

df = pd.read_csv("csv/焼肉きんぐ口コミセンテンス.csv")
filtered_df = df[(df['sentiment'] == 'POSITIVE') & (df['rating'] >= 3.0)]

keywords = ['カルビ', '接客', '席']
sentences_by_keyword = {}

for keyword in keywords:
    keyword_df = filtered_df[filtered_df['sentence'].str.contains(keyword, na=False)]
    keyword_sentences = keyword_df['sentence'].head(10).tolist()
    sentences_by_keyword[keyword] = keyword_sentences

def calculate_cost(model, prompt_tokens, completion_tokens):
    if model == "gpt-4o-mini":
        prompt_cost_per_1M = 0.15  # $0.15 per 1M input tokens
        completion_cost_per_1M = 0.075  # $0.075 per 1M output tokens
    elif model == "gpt-4o":
        prompt_cost_per_1M = 2.5  # $2.5 per 1M input tokens
        completion_cost_per_1M = 1.25  # $1.25 per 1M output tokens
    elif model == "ft:gpt-4o-2024-08-06:mov::B7DJJtvq":
        prompt_cost_per_1M = 3.75  # $2.5 per 1M input tokens
        completion_cost_per_1M = 1.875  # $1.25 per 1M output tokens
    else:
        prompt_cost_per_1M = 0.15
        completion_cost_per_1M = 0.075
    
    # 1Mトークンあたりの価格を計算
    prompt_cost = (prompt_tokens / 1000000) * prompt_cost_per_1M
    completion_cost = (completion_tokens / 1000000) * completion_cost_per_1M
    
    return prompt_cost + completion_cost

def system_prompt():
    prompt = [
        'ある企業の口コミのリストを提供します。',
        '口コミから、利用者に高く評価されているポイントをひとつだけ抽出し、それをPRする投稿を作成してください。',
        '投稿文は、MEO対策を考慮し、"出力ルール"に従って作成してください。\n',
        '# 出力ルール',
        '- 文章は日本語',
        '- 100文字以上200文字以内',
        '- タイトル不要',
        '- 文頭、文末に括弧不要、文章のみ出力',
    ]
    return "\n".join(prompt)

def generate_post(keyword, sentences, model="ft:gpt-4o-2024-08-06:mov::B7DJJtvq"):
    sentences_text = "\n".join([f"- {sentence}" for sentence in sentences])
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt()},
            {"role": "user", "content": sentences_text},
        ],
    )

    post_text = completion.choices[0].message.content

    prompt_tokens = completion.usage.prompt_tokens
    completion_tokens = completion.usage.completion_tokens
    cost = calculate_cost(model, prompt_tokens, completion_tokens)
    
    return post_text, cost

results = []

for keyword, sentences in sentences_by_keyword.items():
    for i in range(10):
        post_text, cost = generate_post(keyword, sentences)
        
        results.append({
            "調整方法": "ファインチューニング",
            "口コミテーマ": keyword,
            "投稿文": post_text,
            "コスト": cost
        })

results_df = pd.DataFrame(results)
results_df.to_csv("csv/投稿文自動生成スタイル調整_ファインチューニング.csv", index=False)

results_df

,調整方法,口コミテーマ,投稿文,コスト
0,未調整,カルビ,『焼肉きんぐ』では、「きんぐカルビはジューシーで最高！」「肉肉しくてうまい！」と多くのお客様...,0.001678
1,未調整,カルビ,『焼肉きんぐ』では「きんぐコース」をご注文いただくと、「【名物】大判 カルビ」が食べ放題！\...,0.001658
2,未調整,カルビ,『焼肉きんぐ』の看板メニューの「きんぐカルビ」が、柔らかくて脂の甘みをしている！と大変ご好評...,0.001536
3,未調整,カルビ,『焼肉きんぐ』の「きんぐカルビ」は、ボリュームたっぷりでジューシーと大好評！きんぐコースや五...,0.001618
4,未調整,カルビ,「焼肉きんぐ」でお気に入りのお肉といえば？\n\n 焼き肉を食べて元気100倍！ そんなとき...,0.001549
5,未調整,カルビ,『焼肉きんぐ』の炙りすき焼きカルビ、もう食べていただけましたか？🤤\n甘辛なタレと、とろ〜り...,0.001532
6,未調整,カルビ,『焼肉きんぐ』自慢の「きんぐカルビ」は脂の甘みがたっぷり楽しめると好評！肉の旨みがギュッと詰...,0.001586
7,未調整,カルビ,『焼肉きんぐ』の「きんぐカルビ」は、食べ放題の焼肉とは思えない、肉肉しくて高いクオリティが人...,0.001686
8,未調整,カルビ,『焼肉きんぐ』の人気の「きんぐカルビ」は、肉肉しくてジューシーな味わいが高評価！他のお肉やサ...,0.001553
9,未調整,カルビ,じゅわっと美味しい『焼肉きんぐの看板商品』、ぜひお試しください！\n\n『きんぐコース』『フ...,0.001686


# exampleを渡す

In [58]:
import pandas as pd

df = pd.read_csv("csv/焼肉きんぐ口コミセンテンス.csv")
filtered_df = df[(df['sentiment'] == 'POSITIVE') & (df['rating'] >= 3.0)]

keywords = ['カルビ', '接客', '席']
sentences_by_keyword = {}

for keyword in keywords:
    keyword_df = filtered_df[filtered_df['sentence'].str.contains(keyword, na=False)]
    keyword_sentences = keyword_df['sentence'].head(10).tolist()
    sentences_by_keyword[keyword] = keyword_sentences

def calculate_cost(model, prompt_tokens, completion_tokens):
    if model == "gpt-4o-mini":
        prompt_cost_per_1M = 0.15  # $0.15 per 1M input tokens
        completion_cost_per_1M = 0.075  # $0.075 per 1M output tokens
    elif model == "gpt-4o":
        prompt_cost_per_1M = 2.5  # $2.5 per 1M input tokens
        completion_cost_per_1M = 1.25  # $1.25 per 1M output tokens
    else:
        prompt_cost_per_1M = 0.15
        completion_cost_per_1M = 0.075
    
    # 1Mトークンあたりの価格を計算
    prompt_cost = (prompt_tokens / 1000000) * prompt_cost_per_1M
    completion_cost = (completion_tokens / 1000000) * completion_cost_per_1M
    
    return prompt_cost + completion_cost

def adjusted_system_prompt():
    prompt = [
        'ある企業の口コミのリストを提供します。',
        '口コミから、利用者に高く評価されているポイントをひとつだけ抽出し、それをPRする投稿を作成してください。',
        '投稿文は、MEO対策を考慮し、"出力ルール"、"言い回しの注意"に従って作成してください。\n',
        '# 出力ルール',
        '- 文章は日本語',
        '- 100文字以上200文字以内',
        '- タイトル不要',
        '- 文頭、文末に括弧不要、文章のみ出力\n',
        '# 言い回しの注意',
        '- 口コミで言及された点を羅列するだけの不自然な投稿文にしないこと',
        '- 投稿文全体がひとつのテーマに沿った自然な構成とすること',
        '- 主観的な定性的評価は書かず、そういった口コミや感想をいただいたという事実のみ記載すること',
        '- 「工夫の結果、サービスの向上につながった」という言い回しを避け、「サービス向上のため、工夫をしている」という言い回しにすること',
        '  - 悪い例: 「テキパキとした接客と丁寧な説明でお客様に快適な時間をご提供しています」',
        '  - 良い例: 「お客様に快適な時間をご提供するために、テキパキとした接客と丁寧な説明を徹底しています」',
    ]
    return "\n".join(prompt)

# 質疑応答例1
user_message1 = """
- 3連休だったので家族と遠出した帰りに一緒に利用しました！サイドメニューが豊富なので、子供も大人も大満足でした！
- 食べ放題コースの五大名物と期間限定メニューがおすすめです！デザートも充実しているので、小さなお子様のいる家庭にも良いと思います。"""

assistant_message1 = """. 
3連休は『焼肉きんぐ』で家族や友人と楽しいひとときを！
選べる食べ放題コースでは、自慢の五大名物や期間限定メニューをはじめ、サラダやデザートも充実。小さなお子様から大人まで楽しめる豊富なサイドメニューが揃い、みんなが満足できること間違いなしです。広々とした席でゆったり過ごせるので、子連れの方にも安心。
この3連休はランチやディナーに、美味しさと笑顔があふれる『焼肉きんぐ』で素敵なひとときをお楽しみください！"""

# 質疑応答例2
user_message2 = """
- お肉はやっぱり名物の「花咲き牛たん」が最高だったけど、カルビ、ハラミ、ホルモンも美味しかった！ランチは特におすすめで、この肉質と量であの値段はおかしい。
- サイドメニューのまぼろし炙りたんユッケ(低温調理)、〆のひとくち生冷麺が大好きでリピートしています！"""

assistant_message2 = """
厳選牛肉をコスパ良く味わいたいなら！
焼き肉は名物の「花咲き牛たん」など豊富な牛たんを使ったメニューの他、カルビ、ハラミ、ホルモンなども取り揃えております。
「まぼろし炙りたんユッケ(低温調理)」「〆のひとくち生冷麺」などサイドメニューも豊富で、居酒屋のようにご利用いただけます。
ディナーはもちろん「この肉の質とボリュームでこの値段は安い！」と大人気のランチもおすすめ。
"""

def generate_adjusted_post(keyword, sentences, model="gpt-4o"):
    sentences_text = "\n".join([f"- {sentence}" for sentence in sentences])
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": adjusted_system_prompt()},
            # 質疑応答例1
            {"role": "user", "content": user_message1.lstrip("\n")},
            {"role": "assistant", "content": assistant_message1.lstrip("\n")},
            # 質疑応答例2
            {"role": "user", "content": user_message2.lstrip("\n")},
            {"role": "assistant", "content": assistant_message2.lstrip("\n")},
            # 本番
            {"role": "user", "content": sentences_text},
        ],
    )

    post_text = completion.choices[0].message.content

    prompt_tokens = completion.usage.prompt_tokens
    completion_tokens = completion.usage.completion_tokens
    cost = calculate_cost(model, prompt_tokens, completion_tokens)
    
    return post_text, cost

adjusted_results = []

for keyword, sentences in sentences_by_keyword.items():
    for i in range(10):
        post_text, cost = generate_adjusted_post(keyword, sentences)
        
        adjusted_results.append({
            "調整方法": "プロンプト調整+example",
            "口コミテーマ": keyword,
            "投稿文": post_text,
            "コスト": cost
        })

adjusted_results_df = pd.DataFrame(adjusted_results)
adjusted_results_df.to_csv("csv/投稿文自動生成スタイル調整_プロンプトと例.csv", index=False)

adjusted_results_df

,調整方法,口コミテーマ,投稿文,コスト
0,プロンプト調整+example,カルビ,焼肉きんぐの人気メニュー「きんぐカルビ」は、多くのお客様から「大きくて美味しい！」と好評をい...,0.002976
1,プロンプト調整+example,カルビ,サービス向上のため、きんぐカルビをはじめとする肉質にこだわっています。特にきんぐカルビは多く...,0.002935
2,プロンプト調整+example,カルビ,「焼肉きんぐ」で大好評の「きんぐカルビ」をご堪能ください！食べ放題コースでは、ジューシーで肉...,0.002950
3,プロンプト調整+example,カルビ,サービス向上のため、焼肉きんぐではきんぐカルビを始めとするカルビメニューが大変好評をいただい...,0.002951
4,プロンプト調整+example,カルビ,ジューシーな味わいが自慢の「きんぐカルビ」！\n当店では、食べ放題メニューの中でも特に人気の...,0.002961
5,プロンプト調整+example,カルビ,焼肉食べ放題で迷ったら『きんぐカルビ』をお試しください！多くのお客様からご好評いただいている...,0.002952
6,プロンプト調整+example,カルビ,お客様にご好評いただいている『焼肉きんぐ』の自慢は、ジューシーで肉肉しい『きんぐカルビ』です...,0.002950
7,プロンプト調整+example,カルビ,きんぐカルビファン続出！クオリティと満足度を追求した食べ放題\n焼肉好きにはたまらない当店自...,0.002971
8,プロンプト調整+example,カルビ,『焼肉きんぐ』の食べ放題で人気No.1の「きんぐカルビ」を堪能しよう！ジューシーで食べ応えの...,0.002956
9,プロンプト調整+example,カルビ,焼肉きんぐの「きんぐカルビ」は、お客様からジューシーで食べ応えがあるとご好評をいただいており...,0.002964
